# 데이터 로드 및 확인

In [107]:
# 전체 라이브러리 모아두기
import pandas as pd
import os
import sentencepiece as spm
import json
import re

In [108]:
# 전처리 추가 (시스템 태그 삭제)
def clean_text(text):
    # 시스템 태그 제거
    text = re.sub(r"#@[^#]+#", "", text)
    # 중복된 기호, 공백 정리
    text = re.sub(r"[!@#\$%^&*\(\)\[\]_+=<>?/|\\~`\"';:]{2,}", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def preprocess_dialogues(json_path, save_path=None):
    import pandas as pd

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    pairs = []

    for dialogue in data['data']:
        body = dialogue['body']
        body.sort(key=lambda x: (x['turnID'], x['utteranceID']))
        
        prev_participant = None
        prev_text = ""
        
        for utt in body:
            pid = utt['participantID']
            text = clean_text(utt['utterance'])
            
            if not text:
                continue
            
            if prev_participant and pid != prev_participant:
                # 서로 다른 참여자 간 대화
                pairs.append((prev_text, text))
            
            prev_participant = pid
            prev_text = text
        

    if save_path:
        df = pd.DataFrame(pairs, columns=["input", "response"])
        df.to_csv(save_path, index=False, encoding='utf-8-sig')

    return pairs

In [109]:
# 전처리 실행
train_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_train/[라벨]한국어SNS_train/개인및관계.json", save_path="../data/text_dataset/save_path/train_pairs.csv")
valid_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_valid/[라벨]한국어SNS_valid/개인및관계.json", save_path="../data/text_dataset/save_path/valid_pairs.csv")

In [110]:
# 학습 샘플 확인
for i in range(20):
    print(f"Q: {train_pairs[i][0]}")
    print(f"A: {train_pairs[i][1]}")
    print("-" * 30)

Q: 잉ㅜㅜ
A: 돈따스
------------------------------
Q: 돈따스
A: 안보내줫어?
------------------------------
Q: 이거
A: 하 ......ㅡ
------------------------------
Q: 퀵으로한대서 두시까지오래 ㅋㅋㅋㅋ
A: ㅎㅎㅎㅎ오좋겠네
------------------------------
Q: ㅎㅎㅎㅎ오좋겠네
A: 잘잣어ㅋㅋㅋㅋㅋ
------------------------------
Q: 잘잣어ㅋㅋㅋㅋㅋ
A: ㅋㄱㅋㄱㄱㄱㄱ아니
------------------------------
Q: 머거
A: 잉
------------------------------
Q: 내돈가쓰...
A: ㅋㄱㄱㄱㄱㄱ맛있어
------------------------------
Q: 고로케도존맛탱
A: 사진찍엇어....
------------------------------
Q: 사진찍엇어....
A: ㅋㄱㄱㄱㅋ
------------------------------
Q: 학생이면좋구!
A: 훔
------------------------------
Q: 없는데...주변에...
A: 왜혼자다니냐고오.....
------------------------------
Q: 왜혼자다니냐고오.....
A: 아니
------------------------------
Q: 어케 친구가있냐..
A: 와 내친군학교나감
------------------------------
Q: 막졸업한애두굳
A: 없다구...
------------------------------
Q: 너무화난당..
A: 흠
------------------------------
Q: 흠
A: 근데오빠는말을또 잘해서 내가화내다보면결국내잘못
------------------------------
Q: 답답해진짱ㅋㅋ
A: 그럴때 억울하지 짖짜
------------------------------
Q: 오빠도 오늘 회식이야?
A: 아니
----------

### train_pairs.csv , valid_pairs.csv -> train.txt로 병합

In [111]:
def merge_csv_to_text(train_csv, valid_csv, output_txt):
    df_train = pd.read_csv(train_csv)
    df_valid = pd.read_csv(valid_csv)
    
    with open(output_txt, 'w', encoding='utf-8') as f:
        for df in [df_train, df_valid]:
            for i in range(len(df)):
                input_text = str(df.loc[i, "input"]).strip()
                response_text = str(df.loc[i, "response"]).strip()
                if input_text and response_text:
                    f.write(input_text + '\n')
                    f.write(response_text + '\n')
                    
merge_csv_to_text(
    "../data/text_dataset/save_path/train_pairs.csv",
    "../data/text_dataset/save_path/valid_pairs.csv",
    "../data/text_dataset/text_for_txt/train.txt"
)

----

# SentencePiece 토크나이저 학습
+ 띄어쓰기/어절 기반이 아닌 서브워드 단위로 토큰 분할
+ 유연하게 희귀 단어 처리 가능
+ 한국어 SNS 데이터에 잘 맞음

In [112]:
def train_sentencepiece(input_file, model_dir="../model/llm_model", model_name="chatbot_spm", vocab_size=16000):
    """
    SentencePiece 모델을 학습하고 지정한 위치에 저장합니다.

    :param input_file: 학습에 사용할 텍스트 파일 경로 (ex: train.txt)
    :param model_dir: 모델과 vocab 파일이 저장될 폴더 경로
    :param model_name: 저장될 모델 파일 이름 접두어
    :param vocab_size: 사용할 vocab 사이즈 (기본: 8000)
    """

    # 저장 경로 포함한 전체 prefix
    model_prefix = os.path.join(model_dir, model_name)

    # SentencePiece 학습 실행
    spm.SentencePieceTrainer.Train(
        f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
        "--model_type=bpe --character_coverage=1.0 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3"
    )

    print(f"✅ 모델 저장 완료: {model_prefix}.model")
    print(f"✅ 단어 사전 저장 완료: {model_prefix}.vocab")

In [117]:
# 실행 예시
train_sentencepiece("../data/text_dataset/text_for_txt/train.txt", model_dir="../model/llm_model", model_name="chatbot_spm")

✅ 모델 저장 완료: ../model/llm_model/chatbot_spm.model
✅ 단어 사전 저장 완료: ../model/llm_model/chatbot_spm.vocab


## Tokenizer 클래스 기능
1. 텍스트 -> 토큰 ID (정수 시퀀스) 변환 (encode)
2. 토큰 ID -> 텍스트 복원 (decode)
3. special token (pad, bos, eos 등) 관리 

In [118]:
class Tokenizer:
    def __init__(self, model_path: str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
        
        self.pad_id = self.sp.pad_id()
        self.unk_id = self.sp.unk_id()
        self.bos_id = self.sp.bos_id()
        self.eos_id = self.sp.eos_id()
    
    def encode(self, text: str, add_bos=True, add_eos=True) -> list:
        tokens = self.sp.encode(text, out_type=int)
        if add_bos:
            tokens = [self.bos_id] + tokens
        if add_eos:
            tokens = tokens + [self.eos_id]
        return tokens
    
    def decode(self, ids: list) -> str:
        ids = [i for i in ids if i not in [self.bos_id, self.eos_id, self.pad_id]]
        return self.sp.decode(ids)
    
    def vacab_size(self):
        return self.sp.get_piece_size()

In [119]:
# 테스트
tokenizer = Tokenizer("../model/llm_model/chatbot_spm.model")

text = "오늘 날씨 좋아?"
encoded = tokenizer.encode(text)
decoded = tokenizer.decode(encoded)

print("✅ 원문:", text)
print("🧠 인코딩:", encoded)
print("🔁 디코딩:", decoded)

✅ 원문: 오늘 날씨 좋아?
🧠 인코딩: [2, 57, 1602, 200, 8898, 3]
🔁 디코딩: 오늘 날씨 좋아?


## LSTM + Attention 모델 정의
+ Embedding Layer : 입력 토큰을 벡터로 변환
+ Encoder : LSTM을 사용해 입력 시퀀스를 인코딩
+ Attention : 디코딩 시 인코더의 중요 부분을 집중해서 학습
+ Decoder : LSTM 디코더 + Atention 출력 결합 + 출력 생성

### Tokenizer가 또 나오는 이유
Because. SentencePiece tokenizer를 감싸는 Tokenizer를 만드는 중이다. <br>
결국 밑에 나오는 코드를 작성해야 DataLoader 구성할 때 input과 response를 숫자 시퀀스로 바꿀 수 있다. <br>
지금 하는 행위는 기존 Transformer 라이브러리의 Tokenizer를 직접 구현하는 중.

In [120]:
# 라이브러리 먼저 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F

In [131]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=False)
        
    def forward(self, x):
        # 단어를 임베팅 벡터로 변환 vocab_size, embed_size
        embeded = self.embedding(x)

        # outputs : 시퀀스 전체 출력 == 모든 시점의 출력(attention용)
        # hidden, cell : 마지막 LSTM hidden state(디코더 초기값)
        outputs, (hidden, cell) = self.lstm(embeded)
        return outputs, (hidden, cell)

# 디코더의 현재 hidden state와 인코더의 전체 output을 비교해 중요한 부분(가중치)를 선택
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()

        # 인코더 출력과 디코더 hidden을 결합 후 처리하는 MLP
        self.attn = nn.Linear(hidden_size * 2, hidden_size)

        # attention score를 계산할 단일 선형 layer
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden : batch, 1, hidden
        # encoder_outputs : batch, seq_len, hidden
        seq_len = encoder_outputs.size(1)
        hidden = hidden.repeat(1, seq_len, 1)

        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        attn_weight = F.softmax(attention, dim=1)

        # context : 현재 시점에서 디코더가 집중해야 할 인코더 출력의 weighted sum
        # attn_weight : 어텐션 가중치 (시각화 기능)
        context = torch.bmm(attn_weight.unsqueeze(1), encoder_outputs)
        return context, attn_weight

# 한 단어씩 예측하면서 문장을 생성하는 역할
# 어텐션을 통해 인코더에서 얻은 context vector 활용
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1, dropout=0.1):
        super(Decoder, self).__init__()

        # 입력 토큰(이전 단어)을 임베딩
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)

        # context vector와 임베딩을 입력으로 받아 다음 hidden state 생성
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size,
                            num_layers, batch_first=True, dropout=dropout)
        
        # 예측된 단어 분포 (vacab 크기만큼 출력)
        self.fc_out = nn.Linear(hidden_size * 2, vocab_size)

        # 인코더 출력과 디코더 hidden을 사용해 어텐션 계산
        self.attention = Attention(hidden_size)

    def forward(self, input_token, hidden, cell, encoder_outputs):
        # 출력 : batch, vocab_size로 각 단어의 확률 분포
        embedded = self.embedding(input_token).unsqueeze(1)

        context, attn_weights = self.attention(hidden[-1].unsqueeze(1), encoder_outputs)
        lstm_input = torch.cat((embedded, context), dim=2)

        # hidden, cell은 다음 시점 디코더에 전달
        outputs, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
        concat = torch.cat((outputs, context), dim=2)
        logits = self.fc_out(concat).squeeze(1)

        return logits, hidden, cell, attn_weights

# 전체 모딜을 연결하여, 인코더 -> 디코더 구조 통합
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.shape
        vocab_size = self.decoder.embedding.num_embeddings

        # outputs : shape[batch_size, trg_len, vocab_size]
        # 이 값을 기준으로 cross entropy loss 계산 가능
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src)
        input_token = trg[:, 0]    # <BOS> 토큰

        for t in range(1, trg_len):
            output, hidden, cell, _ = self.decoder(input_token, hidden, cell, encoder_outputs)
            outputs[:, t] = output

            # teacher_forcing_ratio
            # 1.0 : 항상 정답 토큰을 다음 입력으로 사용
            # 0.0 : 항상 모델이 예측한 값을 다음 입력으로 사용
            # 학습 시 안정성 향상
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

## Tokenizer 기반 학습 데이터셋 구성
ChatDateset 에서 Tokenizer.encode 사용

In [122]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd

class ChatDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len=64):
        df = pd.read_csv(csv_path)
        self.inputs = df["input"].astype(str).tolist()
        self.responses = df["response"].astype(str).tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        src = self.tokenizer.encode(self.inputs[idx])
        trg = self.tokenizer.encode(self.responses[idx])

        # pad
        if len(src) < self.max_len:
            src += [self.tokenizer.pad_id] * (self.max_len - len(src))
        else:
            src = src[:self.max_len]
        
        if len(trg) < self.max_len:
            trg += [self.tokenizer.pad_id] * (self.max_len - len(trg))
        else:
            trg = trg[:self.max_len]
        
        return torch.tensor(src), torch.tensor(trg)
    
# 데이터 로더
train_dataset = ChatDataset("../data/text_dataset/save_path/train_pairs.csv", tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataset = ChatDataset("../data/text_dataset/save_path/valid_pairs.csv", tokenizer)
valid_loader = DataLoader(valid_dataset, batch_size=32)

### LSTM + Attention 챗봇 학습
1. 입력
 + model : Seq2Seq 모델(Encoder + Attention + Decoder)
 + dataloader : 학습 데이터 로더
 + tokenizer : 패딩 ID 확인용
 + num_epochs : 학습 epoch 수
 + lr : 학습률
2. 동작
 + 모델 foward
 + output, target -> reshape
 + CrossEntropyLoss 계산
 + 역전파 + optimizer 업데이트
 + tqdm 진행 표시 및 평균 loss 출력

In [123]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [133]:
def train_model(model, dataloader, tokenizer, num_epochs=5, lr=1e-3, device=None):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # PAD 토큰 무시
    pad_id = tokenizer.pad_id
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for src, trg in progress_bar:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()

            output = model(src, trg)
            output_dim = output.shape[-1]

            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        print(f"\n[Epoch] {epoch+1} 평균 Loss: {epoch_loss / len(dataloader):.4f}\n")
    
    return model

### 모델 저장 및 평가

In [125]:
def save_model(model, path='model/llm_model/chatbot_model.pt'):
    torch.save(model.state_dict(), path)
    print(f"✅ 모델 저장 완료: {path}")

# 모델 불러오기
def load_model(model, path='model/llm_model/chatbot_model.pt'):
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [126]:
from torch.nn.functional import softmax

@torch.no_grad()
def evalute_model(model, dataloader, tokenizer, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.eval()
    model.to(device)

    for i, (src, trg) in enumerate(dataloader):
        if i >= 5: break

        src, trg = src.to(device), trg.to(device)

        # infernce 모드
        output = model(str, trg, teacher_forcing_ratio=0.0)
        pred = output.argmax(dim=-1)

        print("🟢 Input :", tokenizer.decode(src[0].tolist()))
        print("✅ Target :", tokenizer.decode(trg[0].tolist()))
        print("🤖 Output :", tokenizer.decode(pred[0].tolist()))
        print("-"*60)

In [127]:
def generate_reply(model, tokenizer, input_text, max_len=64, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.eval()
    model.to(device)

    input_ids = tokenizer.encode(input_text)
    input_ids = input_ids + [tokenizer.pad_id] * (max_len - len(input_ids))
    src = torch.tensor(input_ids).unsqueeze(0).to(device) # batch=1

    trg = torch.tensor([tokenizer.bos_id] + [tokenizer.pad_id] * (max_len -1)).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(src, trg, teacher_forcing_ratio=0.0)
        pred_ids = output.argmax(dim=-1).squeeze().tolist()
    
    decoded = tokenizer.decode(pred_ids)
    return decoded

### 테스트

In [128]:
def interactive_chat(model, tokenizer, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print("🤖 챗봇 테스트 시작. 종료하려면 exit 입력.")

    while(True):
        query = input("👤 사용자: ")
        if query.strip().lower() in ['exit', 'quit', '종료']:
            print("🔴 테스트 종료.")
            break
        response = generate_reply(model, tokenizer, query, device=device)
        print("🤖 챗봇: ", response)

### 전체 파이프라인

In [129]:
def full_pipeline(train_loader, valid_loader, tokenizer, model, num_epochs=5):
    trained_model = train_model(model, train_loader, tokenizer, num_epochs=num_epochs)

    save_model(trained_model, "model/llm_model/chatbot_model.pt")

    evalute_model(trained_model, valid_loader, tokenizer)

    interactive_chat(trained_model, tokenizer)

### 학습 시작 및 모델 저장

In [135]:
vocab_size = tokenizer.vacab_size()
embed_size = 256
hidden_size = 512

encoder = Encoder(vocab_size, embed_size, hidden_size)
decoder = Decoder(vocab_size, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder, device="cuda" if torch.cuda.is_available() else "cpu")

full_pipeline(train_loader, valid_loader, tokenizer, model, num_epochs=5)

Epoch 1/5:   4%|▎         | 4179/115072 [2:08:27<56:48:47,  1.84s/it, loss=6.54]


KeyboardInterrupt: 